# Extract Dwells from all Samples

Chris Kimmel
7/31/2020

The purpose of this notebook is to extract per-read per-position dwell times from 5 fast5 base-directories (called "fast5 basedirs" in the Tombo framework):
* `/fs/project/PAS1405/GabbyLee/project/m6A_modif/sequencing_data/Trizol_A/Trizol_A_8K_single_fast5`
* `/fs/project/PAS1405/GabbyLee/project/m6A_modif/sequencing_data/Trizol_B/Trizol_B_8K_single_fast5`
* `/fs/project/PAS1405/GabbyLee/project/m6A_modif/sequencing_data/Trizol_C/Trizol_C_8K_single_fast5`
* `/fs/project/PAS1405/GabbyLee/project/m6A_modif/sequencing_data/Trizol_OLD/Trizol_OLD_8K_single_fast5`
* (An IVT control fast5 basedir, the path of which I don't know yet)

All these data files are stored under Gabby Lee's home directory on the OSC Supercomputing Cluster

The dwell-times will be extracted to CSV form - one CSV file per fast5 basedir.
These CSV files will have one row for every read id, and one column for every nucleotide position on the reference genome.
(Nucleotide positions on the reference genome will use *zero-based* indexing.)

This reference genome is currently stored at `/fs/project/PAS1405/General/Kimmel_Chris/RNA_section__454_9627.fa`
(under Chris Kimmel's home directory on the OSC Supercomputing Cluster).
It was used to resquiggle all the fast5 files touched in this notebook.
The column names in the CSV ouput to this notebook are *zero-based* indexes into this reference genome.

In [10]:
import os
import numpy as np
import pandas as pd
import h5py

In [41]:
# define constants

paths = {'A':'/fs/project/PAS1405/GabbyLee/project/m6A_modif/sequencing_data/Trizol_A/Trizol_A_8K_single_fast5',
         'B':'/fs/project/PAS1405/GabbyLee/project/m6A_modif/sequencing_data/Trizol_B/Trizol_B_8K_single_fast5',
         'C':'/fs/project/PAS1405/GabbyLee/project/m6A_modif/sequencing_data/Trizol_C/Trizol_C_8K_single_fast5',
         'OLD':'/fs/project/PAS1405/GabbyLee/project/m6A_modif/sequencing_data/Trizol_OLD/Trizol_OLD_8K_single_fast5',
         'IVT':'/fs/project/PAS1405/GabbyLee/project/m6A_modif/ctrl_data_set/data_Olivier/f1f2_GL'}
reference_path = '/fs/project/PAS1405/General/Kimmel_Chris/RNA_section__454_9627.fa'
# return dwell times from position numbers between min_position (inclusive) and max_position (exclusive)
min_position = 0
max_position = 9300

In [38]:
def get_df_from_basedir(basedir_path):
    '''
    This function returns a pandas dataframe of per-read per-position dwell
    times. The rows are labeled by read ids and the columns are labeled by
    INTEGER position numbers. (Numbers are zero-based indexes into the reference
    genome.)
    
    Args:
        basedir_path (str): filepath to a fast5 basedir
    
    Returns:
        pd.DataFrame: a pandas dataframe of per-read per-position dwell-times
    '''
    fast5_name_list = os.listdir(basedir_path)
    fast5_path_list = [os.path.join(basedir_path, name) for name in os.listdir(basedir_path)]
    
    row_list = []
    weird_occurrences = 0
    for fast5_name, fast5_path in zip(fast5_name_list, fast5_path_list):
        try:
            with h5py.File(fast5_path, 'r') as file:
                # The following lines of code shouldn't be used on other fast5s.
                # At least, not without checking some assumptions.

                # Extract info from fast5
                basecall_subgroup = file['/Analyses/RawGenomeCorrected_000/BaseCalled_template']
                events = basecall_subgroup['Events']
                mapped_start = basecall_subgroup['Alignment'].attrs['mapped_start']
                num_events = len(events)

                # This will become a row in our dataframe
                lengths = events['length']
                event_based_index = np.arange(num_events)
                genome_based_index = event_based_index + mapped_start # ZERO-BASED!
                future_row = (
                    pd.Series(lengths, index=genome_based_index)
                    .rename(fast5_name) # this will become the future row name
                )
                row_list.append(future_row)
        except KeyError:
            pass # This fast5 probably failed resquiggling
        except IOError:
            weird_occurrences += 1
            pass # Sometimes
            
    if weird_occurrences > 1: # allow 1 corrupted file before complaining
        print('Some of the files couldn\'t be opened: ' + str(weird_occurrences))
    
    return pd.concat(row_list, axis=1).T # uses default axis=0, join=outer

display(get_df_from_basedir(paths['A']))


,9,10,11,12,13,14,15,16,17,18,...,9161,9162,9163,9164,9165,9166,9167,9168,9169,9170
13154b6e-f9cc-4e3b-8c2c-5f0fa43de9c2.fast5,NaN,NaN,NaN,NaN,14.0,6.0,19.0,6.0,8.0,6.0,...,67.0,73.0,104.0,64.0,22.0,55.0,26.0,6.0,124.0,NaN
1c498e5b-5e93-4cbe-baf2-8032a2e20ccb.fast5,NaN,NaN,NaN,NaN,NaN,NaN,2.0,9.0,28.0,6.0,...,33.0,66.0,8.0,17.0,6.0,41.0,12.0,6.0,15.0,NaN
ebf6a522-8df0-48a9-a0ef-f4dcdde5339f.fast5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
b76ab61c-db0f-45d5-a65a-b40ce1296342.fast5,NaN,NaN,NaN,NaN,NaN,NaN,6.0,25.0,59.0,6.0,...,6.0,15.0,14.0,30.0,6.0,30.0,51.0,50.0,66.0,12.0
01f25e05-0219-430f-bc26-cb3948023adf.fast5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
e88e4d16-3908-4d11-8c94-fb64ad54266d.fast5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,58.0,39.0,30.0,17.0,7.0,26.0,6.0,NaN,NaN,NaN
92f22b41-79b1-461f-b468-394d5352550d.fast5,NaN,NaN,NaN,NaN,NaN,NaN,6.0,46.0,7.0,6.0,...,25.0,40.0,45.0,19.0,6.0,20.0,13.0,6.0,77.0,15.0
1b050b90-add3-49b1-b6ea-d4c2a488817c.fast5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,23.0,60.0,43.0,15.0,7.0,30.0,25.0,6.0,81.0,NaN
6348393f-6959-403b-a9d5-23b2ad7bd170.fast5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,29.0,104.0,11.0,29.0,6.0,19.0,14.0,6.0,48.0,10.0
2beb0736-f2a1-44ad-9e3b-f8f2a99c940b.fast5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,27.0,37.0,30.0,14.0,6.0,21.0,23.0,164.0,NaN,NaN


In [42]:
# compute dwell times
# This cell took about 60-90 minutes using 28 cores on the Owens cluster

dfs = {key: get_df_from_basedir(path) for key, path in paths.items()}

In [43]:
# write out to a CSV

for key, dframe in dfs.items():
    out_name = ('IVT_lengths.csv' if key == 'IVT'
                else 'Trizol_{}_lengths.csv'.format(key))
    dframe.to_csv(
        out_name,
        sep=',', # default
        na_rep='nan',
        header=True, # default
        index=True, # default
        index_label='read_id',
    )

In [46]:
for key, value in dfs.items():
    print(key, dfs[key].shape)

('A', (398, 9162))
('IVT', (108918, 9170))
('C', (767, 9161))
('B', (660, 9162))
('OLD', (1662, 9164))
